In [ ]:
import pandas as pd

In [ ]:
flight_df = pd.read_csv(r"C:\Users\macie\PROJEKT KOŃCOWY\flight_df_01.csv", delimiter=';')

In [ ]:
flight_df

In [ ]:
airport_list_df = pd.read_csv(r"C:\Users\macie\PROJEKT KOŃCOWY\airport_list.csv", delimiter= ';')
airport_list_df

In [ ]:
airport_list_df['origin_airport_id'].unique()

In [ ]:
airport_list_df['origin_airport_id'].duplicated().unique()

In [ ]:
flight_df.columns

In [ ]:
flight_df = flight_df.merge(airport_list_df[['origin_airport_id', 'origin_city_name']],
                            how = 'left',
                            left_on = 'origin_airport_id',
                            right_on = 'origin_airport_id',
                            suffixes=('_1', '_2'))

In [ ]:
flight_df = flight_df.merge(airport_list_df[['origin_airport_id', 'origin_city_name']],
                            how = 'left',
                            left_on = 'dest_airport_id',
                            right_on = 'origin_airport_id',
                            suffixes=('_3', '_4'))

In [ ]:
flight_df.rename(columns = {'origin_airport_id_4': 'dest_airport_id_4', 'origin_city_name_4': 'destination_city_name', 'origin_city_name_3': 'origin_city_name'}, inplace = True);

## Analiza według lotnisk oraz tras 
1. Wyznaczymy lotniska, z których odlatywało najwięcej samolotów.
2. Wyznaczymy lotniska, na których najwięcej lotów się kończyło
3. Wyznacz najczęściej uczęszczaną trasę
4. Przy założeniu, że reprezentatywna liczba lotów na trasie wynosi ponad 500, wyznacz dodatkowo top 10:  
     - tras z **najmniejszym odsetkiem opóźnień**, wynik zapisz do ramki 
     - tras z **największym odsetkiem opóźnień**, wynik zapisz do ramki 

In [ ]:
top_airports_origin_df = flight_df.groupby(by = 'origin_city_name')['origin_city_name'].count().sort_values(ascending = False)
top_airports_origin_df = pd.DataFrame(top_airports_origin_df)
top_airports_origin_df

In [ ]:
top_airports_origin_df.rename(columns = {'origin_city_name': 'odloty'}, inplace = True)
top_airports_origin_df['origin_city'] = top_airports_origin_df.index
top_airports_origin_df.head(40)

In [ ]:
top_airports_destination_df = flight_df.groupby(by = 'destination_city_name')['destination_city_name'].count().sort_values(ascending = False)
top_airports_destination_df = pd.DataFrame(top_airports_destination_df)
top_airports_destination_df.rename(columns = {'destination_city_name' : 'przyloty'}, inplace = True)
top_airports_destination_df['destination_city'] = top_airports_destination_df.index
top_airports_destination_df

In [ ]:
top_route_df = top_airports_origin_df.merge(top_airports_destination_df,
                                           how = 'inner',
                                           left_on = 'origin_city_name',
                                           right_on = 'destination_city_name')

In [ ]:
top_route_df.drop('destination_city', axis = 1, inplace = True)

In [ ]:
top_route_df['sum'] = 0
o = 0
for i in top_route_df['origin_city']:
    top_route_df.loc[o, 'sum'] = top_route_df.loc[o, 'przyloty'] + top_route_df.loc[o, 'odloty']
    o = o + 1

In [ ]:
top_route_df.sort_values(by = 'sum', ascending = False)

In [ ]:
flight_df = flight_df.loc[:, ~flight_df.columns.duplicated()]


In [ ]:
delayed_flights = flight_df.loc[flight_df['is_delayed'] == 1].groupby(by=['origin_city_name', 'destination_city_name'])['is_delayed'].count()
total_flights = flight_df.groupby(by=['origin_city_name', 'destination_city_name'])['is_delayed'].count()
least_route_delays_df = delayed_flights / total_flights


In [ ]:
least_route_delays_df = pd.DataFrame(least_route_delays_df)
least_route_delays_df.sort_values(by = 'is_delayed', ascending = True).head(10)

In [ ]:
delayed_flights = flight_df.loc[flight_df['is_delayed'] == 1].groupby(by=['origin_city_name', 'destination_city_name'])['is_delayed'].count()
total_flights = flight_df.groupby(by=['origin_city_name', 'destination_city_name'])['is_delayed'].count()
top_route_delays_df = delayed_flights / total_flights

top_route_delays_df = pd.DataFrame(top_route_delays_df)
top_route_delays_df.sort_values(by = 'is_delayed', ascending = False).head(10)

## Wzbogacenie o dane pogodowe 

In [ ]:
airport_weather_df = pd.read_csv(r"C:\Users\macie\PROJEKT KOŃCOWY\airport_weather.csv", delimiter=';')
airport_weather_df.columns = airport_weather_df.columns.str.lower()

In [ ]:
airport_weather_df


In [ ]:
my_columns = ['station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']

airport_weather_df = airport_weather_df[my_columns]

In [ ]:
airport_list_df

In [ ]:
airport_weather_df = airport_weather_df.merge(airport_list_df[['origin_airport_id', 'name']],
                            how = 'inner',
                            left_on = 'name',
                            right_on = 'name')

In [ ]:
airport_weather_df

In [ ]:
airport_weather_df['date']= pd.to_datetime(airport_weather_df['date'])

In [ ]:
flight_df['id'] = flight_df.index
flight_df['date'] = '0'

In [ ]:
p = 0
for q in flight_df['id']:
    flight_df['date'][p] = f"{int(flight_df['year'][p])}-{flight_df['month'][p]}-{flight_df['day_of_month'][p]}"
    p = p + 1

In [ ]:
flight_df

In [ ]:
flight_df['date']= pd.to_datetime(flight_df['date'])
flight_df.rename(columns = {'origin_airport_id_3': 'origin_airport_id'}, inplace = True);


In [ ]:
flight_df = flight_df.merge(airport_weather_df,
                           how = 'left',
                           left_on = ['origin_airport_id', 'date'],
                           right_on =['origin_airport_id', 'date'])

In [ ]:
airport_weather_df

Hipoteza: Temperatura maksymalna wpływa na odsetek opóźnień lotów(kolumna tmax)

 ## Analiza dla kolumny `tmax`

In [ ]:
import plotly.express as px

In [ ]:
delayed_flights_tmax = flight_df.loc[flight_df['is_delayed'] == 1].groupby(by=['tmax'])['is_delayed'].count()
total_flights_tmax = flight_df.groupby(by=['tmax'])['is_delayed'].count()

tmax_research_df = delayed_flights_tmax / total_flights_tmax

In [ ]:
tmax_research_df = pd.DataFrame(tmax_research_df)
tmax_research_df

In [ ]:
fig = px.line(tmax_research_df)
fig.show()

### Wnioski:
Patrząc na dane musimy przede wszystkim ustalić jednostki pomiarowe. Największa wartość w kolumnie tmax to 115, więc to nasuwa wnioski, że używamy pomiaru temperatury w Farenheitach. Największy odsetek opóźnień występuje przy tmax = -10 czyli niska temperatura wpływa na wzrost opóźnień. Jednakże, najmniejsza wartość opóźnienia występuje przy względnie niskiej temperaturze(około -13 stopni celsjusza), więc zważyszwy na to być może występuje dodatkowy czynnik, który kształtuje ten odsetek opóźnień, a którego nie uwzględnialiśmy w naszym badaniu. Stabilizacja następuje w okolicach 40-50 stopniach.

In [ ]:
flight_df.columns

In [ ]:
flight_df_ANOVA = flight_df
flight_df_ANOVA.to_csv("flight_df_ANOVA.csv", sep = ';', encoding = 'utf-8', index = False)